In [1]:
from osgeo import gdal
import numpy as np
import ogr
import geopandas as gpd
import pandas as pd

In [2]:
training = '/Users/arthur.platel/Desktop/Fusion_Images/CZU/PF-SR/2021-06-30.tif'
training_ds= gdal.Open(training)
cols=training_ds.RasterYSize
rows=training_ds.RasterXSize
gdf = gpd.read_file('/Users/arthur.platel/Desktop/Training/TrainingData63021.shp')
class_names = gdf["layer"].unique()
class_ids=gdf['id'].unique()
class_data=gdf['RMSE_1']
print(class_names)
print(class_ids)

['barren' 'burned' 'redwood']
['1' '2' '0']


In [3]:
data=pd.DataFrame({
    'RMSE Blue':[[gdf.iloc[k][2]for k in range(len(gdf))]][0],
    'RMSE Green':[[gdf.iloc[k][3]for k in range(len(gdf))]][0],
    'RMSE Red':[[gdf.iloc[k][4]for k in range(len(gdf))]][0],
    'RMSE Nir':[[gdf.iloc[k][5]for k in range(len(gdf))]][0],
    'RMSE Ndvi':[[gdf.iloc[k][6]for k in range(len(gdf))]][0],
    'landcover':[[gdf.iloc[k][0]for k in range(len(gdf))]][0]
})


In [4]:
print(data.head(5))

    RMSE Blue  RMSE Green    RMSE Red    RMSE Nir  RMSE Ndvi landcover
0   79.078941   81.018654  126.194138  172.050674   0.110265         1
1   92.207298   91.870010  139.448608  271.643463   0.123634         1
2  128.219589  132.448212  208.383545  379.868317   0.164091         1
3  100.595573  101.083458  130.614700  136.000275   0.108531         1
4   64.786537   66.525497  116.477188  185.267014   0.102716         1


In [5]:
from sklearn.model_selection import train_test_split

X=data[['RMSE Blue','RMSE Green','RMSE Red','RMSE Nir','RMSE Ndvi']]  # Features
y=data['landcover']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test


In [6]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=1000, oob_score=True)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [7]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)

print(f'Out-of-bag score estimate: {clf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

Out-of-bag score estimate: 0.92
Mean accuracy score: 0.959


id                                                      1
layer                                              barren
RMSE_1                                          92.207298
RMSE_2                                           91.87001
RMSE_3                                         139.448608
RMSE_4                                         271.643463
RMSE_5                                           0.123634
blueCoefs_                                       1.270384
blueCoef_1                                    -236.077652
blueCoef_2                                      -88.98378
GreenCoefs                                       1.340878
GreenCoe_1                                    -172.810089
GreenCoe_2                                     -91.570343
RedCoefs1                                        2.407123
RedCoefs2                                     -227.943344
RedCoefs3                                     -152.932068
NirCoefs_1                                      -3.479419
NirCoefs_2    